# Guide to tHMM

In [14]:
import numpy as np
import scipy.stats as sp

### Synthesizing Cells (not required by the user)

In [15]:
from lineage.CellVar import CellVar as c
from lineage.CellVar import _double

In [16]:
T = np.array([[1.0, 0.0],
              [0.0, 1.0]])
    
parent_state = 1
parent_cell = c(state=parent_state, left=None, right=None, parent=None, gen=1)
left_cell, right_cell = parent_cell._divide(T)

In [17]:
print(left_cell, parent_cell.left)


 Generation: 2, State: 1, Observation: This cell has no observations to report. 
 Generation: 2, State: 1, Observation: This cell has no observations to report.


In [18]:
print(right_cell, parent_cell.right)


 Generation: 2, State: 1, Observation: This cell has no observations to report. 
 Generation: 2, State: 1, Observation: This cell has no observations to report.


## Creating a synthetic lineage (required by the user) "Two State Model"

In [19]:
from lineage.LineageTree import LineageTree
from lineage.StateDistribution import StateDistribution, get_experiment_time

### Creating a lineage and setting the full lineage (unpruned) as the one to be used

The required probabilities are those that define the tree and act of state switching. This process works by first creating a hidden tree of empty cells. Empty cells are those that have their states set but do not have any observations attached to them. We then draw as many observations from each state distribution and assign those observations to those cells. The $\pi$ and $T$ parameters are easy to define. The number of states is $k$. We require for $\pi$ a $k\times 1$ list of probabilities. These probabilities must add up to $1$ and they should be either in a $1$-dimensional list or a $1$-dimensional numpy array. The $T$ parameter should be a square numpy matrix of size $k\times k$. The rows are the states in which we are transitioning from and the columns are the states in which we are transitioning to. Each row of $T$ should sum to $1$. The columns need not sum to $1$.

In [20]:
# pi: the initial probability vector
pi = np.array([0.95, 0.05], dtype="float")

# T: transition probability matrix
T = np.array([[0.85, 0.15],
              [0.10, 0.90]])

The emission matrix $E$ is a little more complicated to define because this is where the user has complete freedom in defining what type of observation they care about. In particular, the user has to first begin with defining what observation he or she will want in their cells in their synthetic images. For example, if one is observing kinematics or physics, they might want to use Gaaussian distribution observations. In defining the random variables, the user will pull from a Gaussian distribution based on the mean and standard deviation of the different states he or she picks. They can also utilize the Gaussian probability distribution to define the likelihood as well. Furthermore, they can build an analytical estimator for their state distributions that yield the parameter estimates when given a list of observations. Finally, the user can also define a prune rule, which is essentially a boolean function that inspects a cell's observations and returns True if the cell's subtree (all the cells that are related to the cell in question and are of older generation) is to be pruned or False if the cell is safe from pruning. In the Gaussian example, a user can remove a cell's subtree if its observation is higher or lower than some fixed value.

We have already built, as an example, and as bioengineers, a model that resembles lineage trees. In our synthetic model, our emissions are multivariate. This first emission is a Bernoulli observation, $0$ implying death and $1$ implying division. The second and third emissions are continuous and are from exponential and gamma distributions respectively. Though these can be thought of cell lifetime's or periods in a certain cell phase, we want the user to know that these values can really mean anything and they are completely free in choosing what the emissions and their values mean. We define ways to calculate random variables for these multivariate observations and likelihoods of an observations. We also provide as a prune rule, keeping with the cell analogy, that if a cell has a $0$ in its Bernoulli observation, then its subtree is pruned from the full lineage tree. Though this will obviously introduce bias into estimation, we keep both the full tree and the pruned tree in the lineage objects, in the case a user would like to see the effects of analyzing on one versus the other.

Ultimately, $E$ is defined as a $k\times 1$ size list of state distribution objects. These distribution objects are rich in what they can already do, and a user can easily add more to their functionality. They only need to be instantiated by what parameters define that state's distribution.

In [21]:
# E: states are defined as StateDistribution objects

# State 0 parameters "Resistant"
state0 = 0
bern_p0 = 0.99
expon_scale_beta0 = 20
gamma_a0 = 5.0
gamma_scale0 = 1.0

# State 1 parameters "Susciptible"
state1 = 1
bern_p1 = 0.8
expon_scale_beta1 = 80
gamma_a1 = 10.0
gamma_scale1 = 2.0

state_obj0 = StateDistribution(state0, bern_p0, expon_scale_beta0, gamma_a0, gamma_scale0)
state_obj1 = StateDistribution(state1, bern_p1, expon_scale_beta1, gamma_a1, gamma_scale1)

E = [state_obj0, state_obj1]

The final required parameters are more obvious. The first is the desired number of cells one would like in their full unpruned lineage tree. This can be any number. The lineage tree is built 'from left to right'. What this means is that, we construct the binary tree by going to the left-most cell, dividing then walking through the generation. For example, if someone requested for

In [22]:
desired_num_cells = 2**11 - 1 
prune_boolean = True # To get the full tree

In [23]:
lineage1 = LineageTree(pi, T, E, desired_num_cells, prune_boolean)
print(lineage1)

This tree is pruned. It is made of 2 states.
 For each state in this tree: 
 	 There are 515 cells of state 0, 
 	 There are 348 cells of state 1.
 This pruned tree has 863 cells in total


### Obtaining how long the experiment ran by checking the time length of the longest branch

In [24]:
longest_branch_time = get_experiment_time(lineage1)
print(longest_branch_time)

486.38421966792566


### Estimation of distribution parameters using our estimators for full lineage

In [25]:
for state in range(lineage1.num_states):
    print("State {}:".format(state))
    print("                    estimated state", E[state].estimator(lineage1.lineage_stats[state].full_lin_cells_obs))
    print("original parameters given for state", E[state])

State 0:
                    estimated state State object w/ parameters: 0.98989898989888, 19.36960745172329, 4.713553931628664, 1.063025183436546.
original parameters given for state State object w/ parameters: 0.99, 20, 5.0, 1.0.
State 1:
                    estimated state State object w/ parameters: 0.7776816608996059, 79.49491010695701, 9.74329661279069, 2.077273117853873.
original parameters given for state State object w/ parameters: 0.8, 80, 10.0, 2.0.


### Analyzing our first full lineage

In [26]:
from lineage.Analyze import Analyze

X = [lineage1] # population just contains one lineage

deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 2) # find two states

In [27]:
for num, lineageObj in enumerate(X):
    lin_estimated_states = all_states[num]
    lin_true_states = [cell.state for cell in lineageObj.output_lineage]
    total = len(lin_estimated_states)
    assert total == len(lin_true_states)
    counter = [1 if a==b else 0 for (a,b) in zip(lin_estimated_states,lin_true_states)]
    print("Accuracy or 1-Accuracy is {}".format(sum(counter)/total))

Accuracy or 1-Accuracy is 0.03592120509849363


### Estimated Markov parameters ($\pi$, $T$, $E$)

In [28]:
print(tHMMobj.estimate.pi)

[1.00000000e+00 2.75291638e-10]


In [29]:
print(tHMMobj.estimate.T)

[[0.85665368 0.14334632]
 [0.17175213 0.82824787]]


In [30]:
for state in range(tHMMobj.numStates):
    print(tHMMobj.estimate.E[state])

State object w/ parameters: 0.7416666666665325, 75.32775054251123, 8.240990616426801, 2.3929226442424625.
State object w/ parameters: 0.9999999999998013, 19.620818059171675, 5.136181716879978, 0.9714166858282474.


## Trying another lineage, this time pruning branches with ancestors that die

In [31]:
desired_num_cells = 2**12 -1 
prune_boolean = True # To get pruned tree

In [32]:
lineage2 = LineageTree(pi, T, E, desired_num_cells, prune_boolean)
print(lineage2)

This tree is pruned. It is made of 2 states.
 For each state in this tree: 
 	 There are 1497 cells of state 0, 
 	 There are 980 cells of state 1.
 This pruned tree has 2477 cells in total


In [33]:
longest2 = get_experiment_time(lineage2)
print(longest2)

893.2197445047734


### Estimation of distribution parameters using our estimators for pruned lineage

In [34]:
for state in range(lineage2.num_states):
    print("State {}:".format(state))
    print("                    estimated state", E[state].estimator(lineage2.lineage_stats[state].pruned_lin_cells_obs))
    print("original parameters given for state", E[state])

State 0:
                    estimated state State object w/ parameters: 0.9886439545757529, 20.077918499735816, 5.3174302470278345, 0.9426374814579762.
original parameters given for state State object w/ parameters: 0.99, 20, 5.0, 1.0.
State 1:
                    estimated state State object w/ parameters: 0.801020408163204, 85.31071108067567, 10.071696040579692, 1.972151353769305.
original parameters given for state State object w/ parameters: 0.8, 80, 10.0, 2.0.


### Analyzing a population of lineages

In [35]:
X = [lineage1, lineage2] # population just contains one lineage

deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 2) # find two states

In [36]:
for num, lineageObj in enumerate(X):
    lin_estimated_states = all_states[num]
    lin_true_states = [cell.state for cell in lineageObj.output_lineage]
    total = len(lin_estimated_states)
    assert total == len(lin_true_states)
    counter = [1 if a==b else 0 for (a,b) in zip(lin_estimated_states,lin_true_states)]
    print("Accuracy or 1-Accuracy is {}".format(sum(counter)/total))

Accuracy or 1-Accuracy is 0.9826187717265353
Accuracy or 1-Accuracy is 0.9923294307630198


### Estimated Markov parameters ($\pi$, $T$, $E$)

In [37]:
print(tHMMobj.estimate.pi)

[1.0000000e+00 9.0834485e-32]


In [38]:
print(tHMMobj.estimate.T)

[[0.85714411 0.14285589]
 [0.08683975 0.91316025]]


In [39]:
for state in range(tHMMobj.numStates):
    print(tHMMobj.estimate.E[state])

State object w/ parameters: 0.9890656063617804, 19.947511866467472, 5.219414475848831, 0.9619080517204343.
State object w/ parameters: 0.7868975903614026, 83.21838001272671, 9.89289845561557, 2.0153086575009462.


## Creating a synthetic lineage that has three states

Here we generate a lineage with three states, which would be 1) Susciptible 2) Middle State 3) Resistant. The aim here is to show the transition from susciptible to resistant state doesn't happen immediately, and there is a gradual transition which is modeled as a middle state. The point to be considered here is that transition from 1 to 3 or otherwise is not possible so the probability of these transitions are zero, and most likely the initial cells are in susciptible state.

**State 1**: Susceptible

**State 2**: Transition state

**State 3**: Resistant state


In [40]:
# pi: the initial probability vector
pi_3 = np.array([0.5, 0.25, 0.25])

# T: transition probability matrix
T_3 = np.array([[0.65, 0.35, 0.00],
                [0.20, 0.40, 0.40],
                [0.00, 0.10, 0.90]])

In [41]:
# E: states are defined as StateDistribution objects

# State 0 parameters "Susciptible"
state0 = 0
bern_p0 = 0.7
expon_scale_beta0 = 20
gamma_a0 = 5.0
gamma_scale0 = 1.0

# State 1 parameters "Middle state"
state1 = 1
bern_p1 = 0.85
expon_scale_beta1 = 60
gamma_a1 = 10.0
gamma_scale1 = 2.0

# State 2 parameters "Resistant"
state2 = 2
bern_p2 = 0.99
expon_scale_beta2 = 80
gamma_a2 = 15.0
gamma_scale2 = 3.0

state_obj0 = StateDistribution(state0, bern_p0, expon_scale_beta0, gamma_a0, gamma_scale0)
state_obj1 = StateDistribution(state1, bern_p1, expon_scale_beta1, gamma_a1, gamma_scale1)
state_obj2 = StateDistribution(state2, bern_p2, expon_scale_beta2, gamma_a2, gamma_scale2)

E_3 = [state_obj0, state_obj1, state_obj2]

In [42]:
desired_num_cells = 2**13 - 1 
prune_boolean = False # To get the full tree

In [43]:
lineage3 = LineageTree(pi_3, T_3, E_3, desired_num_cells, prune_boolean)
print(lineage3)

This tree is NOT pruned. It is made of 3 states.
 For each state in this tree: 
 	 There are 1042 cells of state 0, 
 	 There are 1586 cells of state 1, 
 	 There are 5563 cells of state 2.
 This UNpruned tree has 8191 cells in total


In [44]:
longest3 = get_experiment_time(lineage3)
print(longest3)

788.0640911783548


### Estimation of distribution parameters using our estimators for full lineage (3 state)

In [48]:
for state in range(lineage3.num_states):
    print("State {}:".format(state))
    print("estimated state for full", E_3[state].estimator(lineage3.lineage_stats[state].full_lin_cells_obs))
    print("estimated state for pruned", E_3[state].estimator(lineage3.lineage_stats[state].pruned_lin_cells_obs))
    print("true_____ state", E_3[state])

State 0:
estimated state for full State object w/ parameters: 0.692898272552746, 19.50557123778098, 5.331257448486042, 0.9478675613853463.
estimated state for pruned State object w/ parameters: 0.7371794871791831, 17.8683222755613, 5.985612044487024, 0.8242333080941282.
true_____ state State object w/ parameters: 0.7, 20, 5.0, 1.0.
State 1:
estimated state for full State object w/ parameters: 0.8568726355611151, 59.60594722035066, 9.821780138019205, 2.041906529670673.
estimated state for pruned State object w/ parameters: 0.859315589353339, 57.00510419211896, 9.308517755316373, 2.1775401734149935.
true_____ state State object w/ parameters: 0.85, 60, 10.0, 2.0.
State 2:
estimated state for full State object w/ parameters: 0.9904727664928819, 79.85546032206081, 15.437536499225562, 2.9090404446667035.
estimated state for pruned State object w/ parameters: 0.9926350245498375, 77.72004582688466, 16.02783395072637, 2.788106297018913.
true_____ state State object w/ parameters: 0.99, 80, 15.

### Analyzing a three state lineage

In [47]:
X = [lineage3] # population just contains one lineage

deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 3) # find three states

In [49]:
for num, lineageObj in enumerate(X):
    lin_estimated_states = all_states[num]
    lin_true_states = [cell.state for cell in lineageObj.output_lineage]
    total = len(lin_estimated_states)
    assert total == len(lin_true_states)
    counter = [1 if a==b else 0 for (a,b) in zip(lin_estimated_states,lin_true_states)]
    print("Accuracy {}".format(sum(counter)/total))

Accuracy 0.6911244048345745


### Estimated Markov parameters ($\pi$, $T$, $E$)

In [50]:
print(tHMMobj.estimate.pi)

[6.23171849e-040 5.18650944e-121 1.00000000e+000]


In [51]:
print(tHMMobj.estimate.T)

[[1.61765294e-14 9.80806367e-06 9.99990192e-01]
 [4.05782962e-14 1.15372066e-31 1.00000000e+00]
 [1.05834573e-08 1.84863071e-03 9.98151359e-01]]


In [52]:
for state in range(tHMMobj.numStates):
    print(tHMMobj.estimate.E[state])

State object w/ parameters: 0.9216793893129642, 68.14382696410674, 3.6281963784344273, 9.517909440974668.
State object w/ parameters: 0.9999999999666667, 17.126540119219847, 46.25215701669438, 0.017305833413753326.
State object w/ parameters: 0.9468864468863923, 68.80483336850538, 4.819746674932408, 7.689423870234425.
